In [ ]:
import numpy as np
import pandas as pd
import joblib
import time
import os

# CONFIGURATION
BASE_DIR = r"C:\Users\ayber\OneDrive\Masaüstü\ML"
# Note: Solver uses v2 models now
MODEL_PATH = os.path.join(BASE_DIR, "Models/mlp_baseline_v2.pkl")
SCALER_X_PATH = os.path.join(BASE_DIR, "Models/scaler_x_v2.pkl")
SCALER_Y_PATH = os.path.join(BASE_DIR, "Models/scaler_y_v2.pkl")

# KINEMATICS ENGINE (No changes here)
class IRB2400_Kinematics:
    def __init__(self):
        self.d1 = 0.615
        self.a1 = 0.100
        self.a2 = 0.705
        self.a3 = 0.0315
        self.d4 = 0.755
        self.d6 = 0.1547
        self.offsets = [0, 0, 180, 0, 0, 0] 

    def _dh_matrix(self, theta, d, a, alpha):
        t = np.radians(theta)
        alp = np.radians(alpha)
        c, s = np.cos(t), np.sin(t)
        ca, sa = np.cos(alp), np.sin(alp)
        return np.array([
            [c, -s*ca, s*sa, a*c],
            [s, c*ca, -c*sa, a*s],
            [0, sa,    ca,   d],
            [0, 0,     0,    1]
        ])

    def forward_kinematics(self, joints_deg):
        q = np.array(joints_deg) + np.array(self.offsets)
        T01 = self._dh_matrix(q[0], self.d1, self.a1, -90)
        T12 = self._dh_matrix(q[1], 0,       self.a2, 0)
        T23 = self._dh_matrix(q[2], 0,       self.a3, -90)
        T34 = self._dh_matrix(q[3], self.d4, 0,       90)
        T45 = self._dh_matrix(q[4], 0,       0,       -90)
        T56 = self._dh_matrix(q[5], self.d6, 0,       0)
        T_total = T01 @ T12 @ T23 @ T34 @ T45 @ T56
        return T_total[:3, 3]

    def get_jacobian(self, joints_deg):
        epsilon = 1e-4
        J = np.zeros((3, 6))
        current_pos = self.forward_kinematics(joints_deg)
        for i in range(6):
            q_perturbed = np.array(joints_deg, dtype=float)
            q_perturbed[i] += epsilon
            pos_perturbed = self.forward_kinematics(q_perturbed)
            J[:, i] = (pos_perturbed - current_pos) / epsilon
        return J

    def check_limits(self, joints_deg):
        # Approximate limits
        limits = [(-180, 180), (-100, 110), (-60, 65), (-200, 200), (-120, 120), (-400, 400)]
        for i, (min_l, max_l) in enumerate(limits):
            if not (min_l <= joints_deg[i] <= max_l): return False
        return True

# HYBRID SOLVER V3 (Feature Engineering Support)
class HybridIKSolver:
    def __init__(self):
        print("Loading v2 AI Models (Wide + Sin/Cos)...")
        self.model = joblib.load(MODEL_PATH)
        self.scaler_x = joblib.load(SCALER_X_PATH)
        self.scaler_y = joblib.load(SCALER_Y_PATH)
        self.robot = IRB2400_Kinematics()
        print("Hybrid Engine Ready.")

    def solve(self, target_pose, current_joints, tolerance=0.001, max_iter=50): 
        stats = {'method': 'ML', 'iterations': 0, 'final_error': 0.0, 'time': 0.0, 'valid_limits': True}
        start_time = time.time()

        # STEP 1: PREPARE INPUTS (FEATURE ENGINEERING)
        # target_pose is [x, y, z, yaw, pitch, roll]
        # Model expects: [x, y, z, sin_y, cos_y, sin_p, cos_p, sin_r, cos_r, q1..q6]
        
        x, y, z, yaw, pitch, roll = target_pose
        
        features = np.array([
            x, y, z,
            np.sin(yaw), np.cos(yaw),
            np.sin(pitch), np.cos(pitch),
            np.sin(roll), np.cos(roll),
            *current_joints
        ])
        
        input_scaled = self.scaler_x.transform([features])
        
        # STEP 2: ML PREDICTION
        pred_scaled = self.model.predict(input_scaled)
        delta_deg = self.scaler_y.inverse_transform(pred_scaled)[0]
        q_guess = current_joints + delta_deg
        
        # STEP 3: CHECK ERROR & CORRECT IF NEEDED
        current_pos_xyz = self.robot.forward_kinematics(q_guess)
        target_xyz = target_pose[:3]
        error = np.linalg.norm(current_pos_xyz - target_xyz)
        
        stats['final_error'] = error
        stats['ml_error'] = error
        
        if error > tolerance:
            stats['method'] = 'Hybrid'
            damping = 0.01 
            
            for i in range(max_iter):
                J = self.robot.get_jacobian(q_guess)
                delta_x = target_xyz - current_pos_xyz
                
                lambda_sq = damping**2 * np.eye(3)
                try:
                    J_inv = J.T @ np.linalg.inv(J @ J.T + lambda_sq)
                except np.linalg.LinAlgError:
                    damping *= 10
                    lambda_sq = damping**2 * np.eye(3)
                    J_inv = J.T @ np.linalg.inv(J @ J.T + lambda_sq)

                delta_q_correction = J_inv @ delta_x
                q_new = q_guess + delta_q_correction
                
                pos_new = self.robot.forward_kinematics(q_new)
                error_new = np.linalg.norm(pos_new - target_xyz)
                
                if error_new < error:
                    q_guess = q_new
                    current_pos_xyz = pos_new
                    error = error_new
                    damping *= 0.5 
                else:
                    damping *= 2.0 
                
                damping = np.clip(damping, 1e-4, 10.0)

                if error < tolerance:
                    stats['iterations'] = i + 1
                    break
            
            stats['final_error'] = error
            
        stats['time'] = (time.time() - start_time) * 1000 
        stats['valid_limits'] = self.robot.check_limits(q_guess)
        
        return q_guess, stats

if __name__ == "__main__":
    # Test on new v2 data to ensure compatibility
    # Note: We need to load raw data to get yaw/pitch/roll for the 'solve' input format
    # The processed v2 file has sin/cos, so we can't use it directly for the TEST loop here 
    # unless we reconstruct angles or load original.
    # EASIER: Load original raw data for testing the full pipeline.
    
    RAW_PATH = os.path.join(BASE_DIR, "Data/external/datasetIRB2400.csv")
    df = pd.read_csv(RAW_PATH).sample(100, random_state=2)
    solver = HybridIKSolver()
    
    print("\nStarting v3 Solver Test...")
    print(f"{'ID':<5} | {'ML Err(mm)':<12} | {'Final(mm)':<10} | {'Time(ms)':<9} | {'Iter'}")
    
    for idx, row in df.iterrows():
        # Raw data is in mm/radians. Solver expects meters/radians for pose, degrees for joints.
        # We must manually prep the test input to match 'solve' expectation
        
        target = np.array([
            row['x']/1000.0, row['y']/1000.0, row['z']/1000.0, 
            row['yaw'], row['pitch'], row['roll']
        ])
        
        current_q = np.degrees(row[['q1_in', 'q2_in', 'q3_in', 'q4_in', 'q5_in', 'q6_in']].values)
        
        q_sol, info = solver.solve(target, current_q, tolerance=0.001, max_iter=50)
        
        if idx % 10 == 0: # Print a few
             print(f"{idx:<5} | {info['ml_error']*1000:<12.2f} | {info['final_error']*1000:<10.4f} | {info['time']:<9.2f} | {info['iterations']}")